## A simple model for demand and supply of publicly-provided services in a city

### Demand modelling

In [ ]:
from enum import Enum
import os.path

import numpy as np
import pandas as pd
import geopandas as gpd
import geopy, geopy.distance
import shapely
from sklearn import gaussian_process

from matplotlib import pyplot as plt 
import seaborn as sns
plt.rcParams['figure.figsize']= (20,14)

In [ ]:
## TODO: find way to put this into some global settings
import os
import sys
nb_dir = os.path.dirname(os.getcwd())
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg

In [ ]:
from src.models.cityItems import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model

In [ ]:
AgeGroup.classify_array(range(18))

In [ ]:
gaussKern = gaussian_process.kernels.RBF
get_random_pos = lambda n: list(map(geopy.Point, list(zip(np.round(np.random.uniform(45.40, 45.50, n), 5), 
                                np.round(np.random.uniform(9.1, 9.3, n), 5)))))
make_shapely_point = lambda geoPoint: shapely.geometry.Point((geoPoint.longitude, geoPoint.latitude))

In [ ]:
### Demand modelling
class DemandUnit:
    def __init__(self, name, position, agesIn, attributesIn={}):
        assert isinstance(name, str), 'Name must be a string'
        assert isinstance(agesIn, dict), 'AgesInput should be a dict'
        assert set(agesIn.keys()) <= set(AgeGroup.all()), 'Ages input keys should be AgeGroups'
        assert isinstance(position, geopy.Point), 'Position must be a geopy Point'
        assert isinstance(attributesIn, dict), 'Attributes can be provided in a dict'
        
        # expand input to all age group keys and assign default 0 to missing ones
        self.ages = {a: agesIn.get(a, 0) for a in AgeGroup.all()}
        self.name = name
        self.position = position
        self.polygon = attributesIn.get('geometry', [])
        
        # precompute export format for speed
        self.export = pd.DataFrame(self.ages, index=(tuple(self.position)))
    
    @property
    def totalPeople(self):
        return sum(self.ages.values())

In [ ]:
def evaluate_demand(householdList, outputServices= [t for t in ServiceType]):
    """ """
    # initialise output
    outDemand = dict()
    # consolidate positions. If two households share the same position, sum components.
    householdData = pd.concat([h.export for h in householdList])
    householdData['position'] = householdData.index 
    consolidated = householdData.groupby('position').sum()
    
    for thisServType in outputServices:
        outDemand[thisServType] = consolidated*thisServType.demandFactors
        
    return outDemand

In [ ]:
class DemandUnitFactory:
    '''
    A class to istantiate DemandUnits
    '''
    def __init__(self, cityNameIn):
        assert cityNameIn in common_cfg.cityList, 'Unrecognised city name "%s"' % cityNameIn
        self.data = common_cfg.get_istat_cpa_data(cityNameIn)
        self.nSections = self.data.shape[0]
        # prepare the AgeGroups cardinalities
        groupsCol = 'ageGroup'
        peopleBySampleAge = common_cfg.fill_sample_ages_in_cpa_columns(self.data)
        dataByGroup = peopleBySampleAge.rename(AgeGroup.find_AgeGroup, axis='columns').T
        dataByGroup.index.name = groupsCol # index is now given by AgeGroup items
        dataByGroup = dataByGroup.reset_index() # extract to convert to categorical and groupby
        dataByGroup[groupsCol] = dataByGroup[groupsCol].astype('category')
        # finally assign in dict form where data.index are the keys
        self.peopleByGroup = dataByGroup.groupby(groupsCol).sum().to_dict()
        
    def make_units_at_centroids(self):
        unitList = []

        # make units
        for iUnit in range(self.nSections):
            rowData = self.data.iloc[iUnit,:]
            sezId = self.data.index[iUnit]
            attrDict = {'geometry':rowData['geometry']}
            # get polygon centroid and use that as position
            sezCentroid =rowData['geometry'].centroid
            sezPosition = geopy.Point(sezCentroid.y, sezCentroid.x)
            
            thisUnit = DemandUnit(name=sezId, 
                        position=sezPosition, 
                        agesIn=self.peopleByGroup[sezId], 
                        attributesIn=attrDict)
            
            unitList.append(thisUnit)
        
        return unitList

In [ ]:
milanDemandFactory = DemandUnitFactory('Milano')

In [ ]:
tt = milanDemandFactory.make_units_at_centroids()
# total number of people modelled
sum(tt[0].ages.values())